<a href="https://www.kaggle.com/code/hossamahmedsalah/icr-preprocessing-the-data?scriptVersionId=137643151" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train = pd.read_csv('/kaggle/input/icr-integer-data/train_integerized.csv')
greek = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

# Combining train set with greeks set

In [3]:
train = pd.merge(train, greek, on='Id')
train

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,49.0,1284724.5,9783.0,7231.0,1552.0,79.0,42.0,1557.0,790.5,...,458118.5,5949.0,27152.0,0.120343,1,B,C,G,D,3/19/2019
1,007255e47698,34.0,404448.0,9783.0,11937.0,1552.0,410.0,42.0,2145.0,175.0,...,6397248.0,7830.0,12493.0,21.978000,0,A,C,M,B,Unknown
2,013f2bd269f5,110.0,1088887.0,9783.0,10449.0,1552.0,760.0,42.0,2035.0,175.0,...,3126876.5,7531.0,13683.0,0.196941,0,A,C,M,B,Unknown
3,043ac50845d5,59.0,1578368.5,13802.0,24899.0,1552.0,416.0,42.0,1754.0,175.0,...,478798.0,10736.0,35184.0,0.155829,0,A,C,M,B,Unknown
4,044fb8a146ec,89.0,1542582.0,9783.0,4554.0,1552.0,445.0,90.0,539.0,14535.0,...,1948873.0,12196.0,14099.0,0.096614,1,D,B,F,B,3/25/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,35.0,1293413.0,14211.0,3072.0,2483.0,395.0,127.0,1356.0,399.0,...,1850922.0,6622.0,26902.0,21.978000,0,A,B,M,B,9/13/2020
613,fd895603f071,102.0,2257039.0,9783.0,15031.0,3046.0,675.0,42.5,2003.0,537.5,...,705374.5,7968.0,48526.0,0.145340,0,A,B,M,B,9/8/2020
614,fd8ef6377f76,100.0,1016160.0,14943.0,17874.0,1908.0,911.0,42.0,2445.0,175.0,...,1480259.0,7032.0,46485.0,21.978000,0,A,C,M,B,7/24/2019
615,fe1942975e40,85.0,522122.0,9783.0,7648.0,1552.0,901.0,42.0,1194.0,175.0,...,449324.0,6750.0,14446.0,0.184622,0,A,C,M,B,1/31/2019


In [4]:
train.columns

Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class', 'Alpha',
       'Beta', 'Gamma', 'Delta', 'Epsilon'],
      dtype='object')

# Preprocessing and feature engineering

In [5]:
def nullReport(dataset):
    for col in dataset.columns:
        na = dataset[col].isnull().sum()
        if na != 0:
            print(col, na)


In [6]:
nullReport(train)

BQ 60
CB 2
CC 3
DU 1
EL 60
FC 1
FL 1
FS 2
GL 1


In [7]:
cols_missings = ['BQ', 'CB', 'CC', 'DU', 'EL', 'FC', 'FL', 'FS', 'GL']

In [8]:
grouped_data = train.groupby('Class')[cols_missings].agg(['mean', 'std'])
grouped_data

BQ                         CB                           CC  \
               mean           std         mean           std           mean   
Class                                                                         
0      32641.946548  33465.157654  8507.022682  18338.057561  113632.816568   
1      58966.435185  43298.823554  7835.476852  11302.161570  107415.051402   

                              DU                           EL                \
                std         mean           std           mean           std   
Class                                                                         
0      34972.277913   519.440945   1144.041338  159308.471429  90170.929091   
1      69929.292304  5011.546296  14952.316476  175083.156863  87813.238369   

                 FC                       FL                   FS              \
               mean           std       mean        std      mean         std   
Class                                                                           
0      12082.280943  21608.757213   4.139887   9.217830  62.13189  209.705082   
1      14428.158879  51296.546274  11.516557  17.655941  62.71028   69.440895   

             GL             
           mean        std  
Class                       
0      9.098095  10.398146  
1      5.863328   9.588968

In [9]:
for col in cols_missings:
    missing_mask = train[col].isnull()
    group_missing = train.groupby([missing_mask, 'Class'])[['Id']].count()
    print(f"Grouped data for {col}:")
    display(group_missing)

Grouped data for BQ:


Id
BQ    Class     
False 0      449
      1      108
True  0       60

Grouped data for CB:


Id
CB    Class     
False 0      507
      1      108
True  0        2

Grouped data for CC:


...

Id
FL    Class     
False 0      508
      1      108
True  0        1

Grouped data for FS:


Id
FS    Class     
False 0      508
      1      107
True  0        1
      1        1

Grouped data for GL:


Id
GL    Class     
False 0      508
      1      108
True  0        1

-  we see with ```BQ``` all the missing values are class 0
- ```CB``` the 2 missing valuess for class 0
- ```CC``` 2 missing class 0 and 1 is class 1
- ...and so on
- ```EL``` 54 class 0 and 6 class 1 
> so most of the missing values with class 0 who has no Age related conditions ARC


# Encodeing and feature engineering 

In [10]:
# before we solve the problem with missing values I will encode categories to build a model that predict the missing values of the two columns
train.describe(include='O')

,Id,EJ,Alpha,Beta,Gamma,Delta,Epsilon
count,617,617,617,617,617,617,617
unique,617,2,4,3,8,4,198
top,000ff2bfdfe9,B,A,C,M,B,Unknown
freq,1,395,509,407,445,456,144


In [11]:
train.describe(include='O').columns

Index(['Id', 'EJ', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon'], dtype='object')

In [12]:
cats = ['EJ', 'Alpha', 'Beta', 'Gamma', 'Delta']
for cat in cats:
    train[cat] = train[cat].astype('category').cat.codes

In [13]:
train.describe(include='O').columns
# only 'Epsilon' to deal with now 

Index(['Id', 'Epsilon'], dtype='object')

In [14]:
# 'Epsilon' maybe droping it is better!
train.drop(columns=['Epsilon'], inplace=True)

In [15]:
# for all small missing values I would just use the mean 
for col in cols_missings:
    if col not in ['BQ', 'EL']:
        train[col] = train[col].fillna(train[col].mean())


In [16]:
nullReport(train)

BQ 60
EL 60


In [17]:
cols_missings = ['BQ', 'EL']

In [18]:
train[train[cols_missings].isna().any(axis=1)]['BQ']
# a note 67 not 60 as this code show take the null values if any exist in BQ or EL the two 

8     NaN
15    NaN
18    NaN
23    NaN
45    NaN
       ..
583   NaN
594   NaN
602   NaN
603   NaN
615   NaN
Name: BQ, Length: 67, dtype: float64

# Predicting the missing values uing a simple random forest

In [19]:
# Separate the data into training and testing sets based on missing values in the missing_features
# train on the data that has no nulls to predict the missing data 
train_data = train[~train[cols_missings].isna().any(axis=1)].copy()
test_data = train[train[cols_missings].isna().any(axis=1)].copy()

# Split the training data into features and targets
X_train = train_data.drop(cols_missings + ['Id'], axis=1)
y_train_bq = train_data['BQ']
y_train_el = train_data['EL']

# Train a random forest regressor to predict BQ
model_bq = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model_bq.fit(X_train, y_train_bq)

# Train a random forest regressor to predict EL
model_el = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model_el.fit(X_train, y_train_el)

# Predict the missing BQ and EL values using the trained models
# dropping the columns won't be needed for predictions 
test_data.loc[:, 'BQ'] = model_bq.predict(test_data.drop(cols_missings + ['Id'], axis=1))
test_data.loc[:, 'EL'] = model_el.predict(test_data.drop(cols_missings + ['Id'], axis=1))

# Merge the training and testing sets back into a single dataset
train = pd.concat([train_data, test_data], axis=0)

In [20]:
nullReport(train)

In [21]:
train.columns

Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class', 'Alpha',
       'Beta', 'Gamma', 'Delta'],
      dtype='object')

In [22]:
train

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,000ff2bfdfe9,49.0,1284724.5,9783.0,7231.0,1552.0,79.0,42.0,1557.0,790.5,...,16439.0,458118.5,5949.0,27152.0,0.120343,1,1,2,4,3
1,007255e47698,34.0,404448.0,9783.0,11937.0,1552.0,410.0,42.0,2145.0,175.0,...,16439.0,6397248.0,7830.0,12493.0,21.978000,0,0,2,6,1
2,013f2bd269f5,110.0,1088887.0,9783.0,10449.0,1552.0,760.0,42.0,2035.0,175.0,...,20061.0,3126876.5,7531.0,13683.0,0.196941,0,0,2,6,1
3,043ac50845d5,59.0,1578368.5,13802.0,24899.0,1552.0,416.0,42.0,1754.0,175.0,...,18659.0,478798.0,10736.0,35184.0,0.155829,0,0,2,6,1
4,044fb8a146ec,89.0,1542582.0,9783.0,4554.0,1552.0,445.0,90.0,539.0,14535.0,...,33079.0,1948873.0,12196.0,14099.0,0.096614,1,2,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,f12d8570682a,66.0,1746353.0,10494.0,4425.0,1552.0,555.0,42.0,1494.0,175.0,...,16439.0,484389.0,5567.0,11581.0,0.069164,0,0,2,6,1
594,f46e39a96ee9,157.0,2273444.5,10773.0,2258.0,1552.0,735.0,42.0,1514.0,175.0,...,32486.0,422851.5,8431.0,3905.0,8.530961,0,0,2,6,1
602,f955c0ed8895,82.0,2244478.5,9783.0,3748.0,1552.0,854.0,42.0,1629.5,340.5,...,48044.0,2091100.5,4646.0,11941.0,0.171600,0,0,2,6,1
603,f96e7ca4a16c,155.0,704297.0,14216.0,22071.0,1552.0,628.0,171.0,1069.0,175.0,...,16439.0,541705.0,5205.0,9950.0,0.130680,0,0,2,6,1


# Scalling the data

In [23]:
train.columns

Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class', 'Alpha',
       'Beta', 'Gamma', 'Delta'],
      dtype='object')

In [24]:
cols_num = [col for col in train.columns if col not in ['Class','Alpha','Beta', 'Gamma', 'Delta', 'Ej', 'Id']]
scaler = MinMaxScaler()
scaler.fit(train[cols_num])
train[cols_num] = scaler.transform(train[cols_num])


In [25]:
train

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,000ff2bfdfe9,0.021082,0.102347,0.000000,0.030632,0.0,0.000000,0.000000,0.180337,0.002958,...,0.000000,0.013846,0.176983,0.362261,0.005425,1,1,2,4,3
1,007255e47698,0.010541,0.027589,0.000000,0.053864,0.0,0.078048,0.000000,0.284500,0.000000,...,0.000000,0.194527,0.274495,0.164135,1.000000,0,0,2,6,1
2,013f2bd269f5,0.063949,0.085715,0.000000,0.046519,0.0,0.160575,0.000000,0.265013,0.000000,...,0.011229,0.095035,0.258994,0.180218,0.008910,0,0,2,6,1
3,043ac50845d5,0.028110,0.127285,0.019180,0.117854,0.0,0.079462,0.000000,0.215235,0.000000,...,0.006882,0.014475,0.425143,0.470820,0.007039,0,0,2,6,1
4,044fb8a146ec,0.049192,0.124246,0.000000,0.017417,0.0,0.086300,0.002841,0.000000,0.069008,...,0.051588,0.059198,0.500829,0.185841,0.004345,1,2,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,f12d8570682a,0.033029,0.141551,0.003393,0.016780,0.0,0.112238,0.000000,0.169176,0.000000,...,0.000000,0.014645,0.157180,0.151808,0.003096,0,0,2,6,1
594,f46e39a96ee9,0.096978,0.186314,0.004725,0.006082,0.0,0.154680,0.000000,0.172719,0.000000,...,0.049749,0.012773,0.305651,0.048062,0.388128,0,0,2,6,1
602,f955c0ed8895,0.044273,0.183855,0.000000,0.013438,0.0,0.182740,0.000000,0.193180,0.000795,...,0.097982,0.063525,0.109435,0.156674,0.007757,0,0,2,6,1
603,f96e7ca4a16c,0.095573,0.053054,0.021155,0.103893,0.0,0.129451,0.007634,0.093888,0.000000,...,0.000000,0.016389,0.138414,0.129764,0.005895,0,0,2,6,1


In [26]:
out = train.to_csv('train_processed0.csv')

In [27]:
train.drop(columns='Id')

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,0.021082,0.102347,0.000000,0.030632,0.0,0.000000,0.000000,0.180337,0.002958,0.047364,...,0.000000,0.013846,0.176983,0.362261,0.005425,1,1,2,4,3
1,0.010541,0.027589,0.000000,0.053864,0.0,0.078048,0.000000,0.284500,0.000000,0.074042,...,0.000000,0.194527,0.274495,0.164135,1.000000,0,0,2,6,1
2,0.063949,0.085715,0.000000,0.046519,0.0,0.160575,0.000000,0.265013,0.000000,0.067011,...,0.011229,0.095035,0.258994,0.180218,0.008910,0,0,2,6,1
3,0.028110,0.127285,0.019180,0.117854,0.0,0.079462,0.000000,0.215235,0.000000,0.048203,...,0.006882,0.014475,0.425143,0.470820,0.007039,0,0,2,6,1
4,0.049192,0.124246,0.000000,0.017417,0.0,0.086300,0.002841,0.000000,0.069008,0.078555,...,0.051588,0.059198,0.500829,0.185841,0.004345,1,2,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,0.033029,0.141551,0.003393,0.016780,0.0,0.112238,0.000000,0.169176,0.000000,0.086654,...,0.000000,0.014645,0.157180,0.151808,0.003096,0,0,2,6,1
594,0.096978,0.186314,0.004725,0.006082,0.0,0.154680,0.000000,0.172719,0.000000,0.047365,...,0.049749,0.012773,0.305651,0.048062,0.388128,0,0,2,6,1
602,0.044273,0.183855,0.000000,0.013438,0.0,0.182740,0.000000,0.193180,0.000795,0.089215,...,0.097982,0.063525,0.109435,0.156674,0.007757,0,0,2,6,1
603,0.095573,0.053054,0.021155,0.103893,0.0,0.129451,0.007634,0.093888,0.000000,0.063388,...,0.000000,0.016389,0.138414,0.129764,0.005895,0,0,2,6,1
